In [ ]:
# !py -m pip install pycalphad
# !py -m pip install numpy
# !py -m pip install tqdm
# !py -m pip install ipywidgets

from pycalphad import Database, equilibrium, variables as v
# from pycalphad.core.utils import filter_phases
import random
import numpy as np
import time
from tqdm import tqdm
# from tqdm.auto import tqdm

# constants:
P = 101325
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)
# TODO: check 'PM' param
all_components = ['LI', 'BE', 'NA', 'MG', 'AL', 'K', 'CA', 'SC', 'TI', 'V', 'CR', 'MN', 'FE',
                  'CO', 'NI', 'CU', 'ZN', 'GA', 'Y', 'ZR', 'NB', 'MO', 'TC',
                  'RH', 'PD', 'AG', 'CD', 'IN', 'SN', 'BA', 'LA', 'CE', 'PR', 'ND', 'SM', 'EU',
                  'GD', 'TB', 'DY', 'HO', 'ER', 'HF', 'TA', 'W', 'RE', 'OS', 'IR',
                  'PT', 'AU', 'HG', 'TL', 'PB', 'BI', 'C', 'SI']
all_phases = list(db.phases)
all_phases.sort()
all_components.sort()

In [44]:
# phase result parser
def flattenArray(arr):
    result = []
    for item in arr.flat:
        if isinstance(item, np.ndarray):
            result.extend(flattenArray(item))
        else:
            result.append(item)
    return result

# Get all possible phases for that components
def getPossiblePhases(db, components):
    possible_phases = []
    for phase_full in db.phases.items():
        # item[0] - name, item[1] - content
        # item[1]: name, constituents, sublattices, model_hints
        found_sublattices = 0
        phase = phase_full[1]

        for sublattice_c in phase.constituents:
            found = False
            for comp in components:
                if len([species.name for species in sublattice_c if species.name == comp]) > 0:
                    found = True
                    break
            if found:
                found_sublattices += 1
            else:
                break

        if found_sublattices == len(phase.constituents):
            possible_phases.append(phase.name)

    possible_phases.sort()
    return possible_phases

# TODO: fix ordered_phases
def filterOrderedPhases(dbf, candidate_phases=None):
    if candidate_phases == None:
        candidate_phases = dbf.phases.keys()
    else:
        candidate_phases = set(
            candidate_phases).intersection(dbf.phases.keys())
    # all_phases = [{"name": phase, "ordered": dbf.phases[phase].model_hints.get('ordered_phase'),'dis':dbf.phases[phase].model_hints.get('disordered_phase')} for phase in candidate_phases if dbf.phases[phase].model_hints.get('ordered_phase') != None]
    # disordered_phases = [dbf.phases[phase].model_hints.get('disordered_phase') for phase in candidate_phases]
    ordered_phases = [dbf.phases[phase].model_hints.get(
        'ordered_phase') for phase in candidate_phases]

    phases = [phase for phase in candidate_phases if
              (phase not in ordered_phases)]
    #  or (phase in disordered_phases and
    # dbf.phases[phase].model_hints.get('ordered_phase') not in candidate_phases)
    return sorted(phases)


def getPhases(int_mask, all_phases=all_phases):
    return [all_phases[i] for i in range(len(all_phases)) if (int_mask & (1 << i)) > 0]


def phasesToMask(phases, all_phases=all_phases):
    phases.sort()
    mask = [1 if phase in phases else 0 for phase in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)


def phaseToMask(phase, all_phases=all_phases):
    mask = [1 if phase == phase_ else 0 for phase_ in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)

def compToMask(components, all_components=all_components):
    components.sort()
    mask = [1 if comp in components else 0 for comp in all_components][::-1]
    return int(''.join(map(str, mask)), 2)

def getComponents(int_mask, all_components=all_components):
    return [all_components[i] for i in range(len(all_components)) if (int_mask & (1 << i)) > 0]

def equilibriumRow(conditions, components, temp, amounts, iter):
    calc_components = components.copy()
    calc_components.append('VA')
    row_values = {}
    founded_phases = []
    possible_phases = getPossiblePhases(db, components)
    possible_phases = filterOrderedPhases(db, possible_phases)
    # print(set(possible_phases) - set(filtred_phase))

    tic = time.perf_counter()
    try:
        eq = equilibrium(db, calc_components, possible_phases,
                         conditions, verbose=False)
        founded_phases = flattenArray(np.array(eq.Phase))
    except ValueError as error:
        row_values['Error'] = error
    toc = time.perf_counter()

    row_values = {'iter': iter, 'T': temp, 'amounts': amounts, 'Components': compToMask(components), 'phases': phasesToMask(founded_phases),
                  'ellapsed_time': toc - tic, 'possible_phases': phasesToMask(possible_phases), 'P': P, 'Error': ''} | row_values

    return row_values

def parseConditions(temp, amounts, components, P=P):
    conditions = {v.T: temp, v.P: P}

    for i in range(len(components)):
        if i != 0:
            conditions[v.X(components[i])] = amounts[i]

    return conditions

def getAmounts(components):
    amounts = [0.05]*len(components)
    for i in range(len(amounts)):
        amounts[i] += random.uniform(0, min(1 - sum(amounts), 0.3))
    amounts[-1] += 1 - sum(amounts)
    return amounts

# TODO: make random temperature more stable diffuse
def getTemp():
    return random.randint(300, 3000)


In [ ]:
import csv
import warnings
warnings.filterwarnings("ignore")

random.seed(42)
# skip empty case
components = random.sample(all_components, 8) # ['RH', 'CA', 'AL', 'TC', 'GA', 'EU', 'ER', 'CD']
# skip one phase case
components = random.sample(all_components, 8) # ['TC', 'C', 'SM', 'ZR', 'OS', 'BI', 'PR', 'MG']
# skip one phase case
components = random.sample(all_components, 8) # ['AU', 'AL', 'BI', 'DY', 'ER', 'ND', 'PT', 'ZN']
line = {}

# 
with open('ParsedData/equilibrium_result.csv', "a") as csv_file:
    header = ['iter', 'T','amounts','Components','phases','ellapsed_time','possible_phases','P', 'Error']
    writer = csv.writer(csv_file, delimiter=';', lineterminator = '\n')
    # writer.writerow(header)
    rnd_state = random.getstate()
    for j in range(100):
        # TODO: iterate over components
        components = random.sample(all_components, 8)
        print(f'{j} - {components}')
        for i in tqdm(range(30)):
            temp = getTemp()
            amounts = getAmounts(components)
            if j <= 19:
                continue
            conditions = parseConditions(temp, amounts, components)
            line = equilibriumRow(conditions, components, temp, amounts, i) 
            writer.writerow(list(line.values()))
    # TODO: Save random state in a file and continue from state
    print('Done!')

0 - ['PB', 'CU', 'TA', 'SC', 'SN', 'OS', 'LI', 'ER']
100%|██████████| 30/30 [00:00<?, ?it/s]
1 - ['DY', 'SM', 'RH', 'FE', 'ND', 'NB', 'W', 'BA']
100%|██████████| 30/30 [00:00<00:00, 30045.16it/s]
2 - ['SN', 'TL', 'HF', 'SM', 'PT', 'CA', 'PD', 'AU']
100%|██████████| 30/30 [00:00<?, ?it/s]
3 - ['SI', 'V', 'Y', 'SC', 'CE', 'NB', 'AU', 'CD']
100%|██████████| 30/30 [00:00<00:00, 29952.18it/s]
4 - ['CE', 'SM', 'NA', 'BE', 'BI', 'V', 'TL', 'MG']
100%|██████████| 30/30 [00:00<?, ?it/s]
5 - ['BA', 'SC', 'NA', 'IR', 'PB', 'C', 'TA', 'NI']
100%|██████████| 30/30 [00:00<00:00, 29980.73it/s]
6 - ['HF', 'NA', 'EU', 'W', 'SM', 'LA', 'ZR', 'MO']
100%|██████████| 30/30 [00:00<?, ?it/s]
7 - ['GD', 'IR', 'RE', 'ER', 'W', 'CD', 'NA', 'CE']
100%|██████████| 30/30 [00:00<?, ?it/s]
8 - ['BA', 'RH', 'AG', 'SN', 'CO', 'BI', 'NB', 'MG']
100%|██████████| 30/30 [00:00<00:00, 29930.81it/s]
9 - ['PB', 'CO', 'NB', 'GD', 'CD', 'CR', 'HG', 'MN']
100%|██████████| 30/30 [00:00<?, ?it/s]
10 - ['BE', 'FE', 'TA', 'PD', 'RH', 'ZR', 'NB', 'MO']
100%|██████████| 30/30 [00:00<00:00, 30009.33it/s]
11 - ['HF', 'TB', 'SN', 'GD', 'CD', 'CR', 'ZN', 'AG']
100%|██████████| 30/30 [00:00<?, ?it/s]
12 - ['PR', 'C', 'AG', 'ND', 'SI', 'IR', 'PB', 'SC']
100%|██████████| 30/30 [00:00<00:00, 29966.45it/s]
13 - ['EU', 'HO', 'GA', 'BA', 'ND', 'ER', 'PB', 'SC']
100%|██████████| 30/30 [00:00<?, ?it/s]
14 - ['SM', 'CA', 'OS', 'AG', 'SC', 'TB', 'RE', 'BE']
100%|██████████| 30/30 [00:00<00:00, 30002.17it/s]
15 - ['AG', 'K', 'SC', 'DY', 'SN', 'HG', 'LI', 'IR']
100%|██████████| 30/30 [00:00<00:00, 29995.02it/s]
16 - ['CR', 'AL', 'CD', 'TB', 'EU', 'SI', 'FE', 'TC']
100%|██████████| 30/30 [00:00<00:00, 29916.58it/s]
17 - ['RH', 'PD', 'IN', 'SM', 'MO', 'SN', 'SI', 'MG']
100%|██████████| 30/30 [00:00<00:00, 30037.99it/s]
18 - ['IR', 'CR', 'K', 'NB', 'NA', 'CD', 'CO', 'CU']
100%|██████████| 30/30 [00:00<00:00, 30066.70it/s]
19 - ['AU', 'MO', 'K', 'TA', 'NI', 'CA', 'AG', 'HF']
100%|██████████| 30/30 [00:00<?, ?it/s]
20 - ['W', 'NI', 'AU', 'HO', 'BI', 'CE', 'PT', 'TL']
100%|██████████| 30/30 [12:48<00:00, 25.60s/it]
21 - ['CA', 'V', 'CE', 'SM', 'NA', 'NB', 'CR', 'CO']
100%|██████████| 30/30 [08:10<00:00, 16.35s/it]
22 - ['ZR', 'MN', 'SM', 'SI', 'BI', 'ND', 'LA', 'CU']
100%|██████████| 30/30 [15:14<00:00, 30.47s/it]
23 - ['BA', 'ZN', 'BE', 'NB', 'SM', 'AL', 'CR', 'K']
 60%|██████    | 18/30 [07:17<05:02, 25.24s/it]